# ArcGIS Online Item Usage Audit

Description: This will search the organization to find views over the specified time period.
 
Created on: 3/16/2020
 
Purpose: As organizations move infrastructure, it is appropriate to gauge how much old infrastructure was used and if it is apprpriate to replace. This script will help with that by providing the number of item views over a specified time. Optionally, through time, various items may be created in an ArcGIS Organization. To ensure items can be made authoritative or deprecated, the usage can be tracked.
 
Authored by: Rick Frantz

# Connect to ArcGIS Online

In [ ]:
# import
from arcgis.gis import GIS
import getpass
from IPython.display import display
from arcgis.mapping import WebMap

# Connection Variables
Organization = input("What's your oranization? ")
User = input("What's your  username? ")
Password = getpass.getpass('Password: ')

# Connection 
gis = GIS(Organization, User, Password, verify_cert=False)
gis
Audit = gis.content.search(query="", max_items = 10000)
#Audit = gis.content.search(query="", item_type="Web Map", max_items = 10000)
auditLen = len(Audit)
if auditLen > 10000:
    print("The maximum number of records has been reached.")
print("Found {} items to asses".format(auditLen))

# Get the date of 30 days ago
from datetime import datetime, timedelta
tday = datetime.now().date()
lastMonth = datetime.today() - timedelta(days=31) # because time is maintained, 31 days in query = 30 days of data

# Create the pandas dataframe
import pandas as pd
dataColumns = ['ItemTitle','ItemID','ItemType','Owner','Year_Views','Month_Views','LastUsed','Last_Searched']
df = pd.DataFrame(columns = dataColumns)
lused = None

# get list of item ids
prevIDs = []

# Past Results to Import?

In [ ]:
# variable for path? user input to get path?

prvdf = pd.read_csv (r'E:/AGO_ItemUsage.csv')

if (df.columns == prvdf.columns).all:

    # code to drop if too old
    prvdf['Last_Searched'] = pd.to_datetime(prvdf['Last_Searched']) # Convert the date to datetime64 
    filtered_df = prvdf.loc[(prvdf['Last_Searched'] >= '2021-01-01')] # Filter data by date

    prevIDs = filtered_df['ItemID'].tolist()

    print("Initially found {} items to asses in AGO".format(auditLen))
    print("A csv containing {} items previously assessed was provided".format(prvdf.shape[0]))
    print("This csv was limited to {} items already recently assessed".format(filtered_df.shape[0]))

    df = filtered_df
    
else:
    print("The columns of the previous csv don't match current columns")

# Audit Section

In [ ]:
# Get the number of views per item

# Find the number of views
for Item in Audit:
    # print("Starting Item {}".format(Item.id))
    if Item.id not in prevIDs:
        # print("The item has not been assessed recently")
        ItemID = Item.id
        gisItem = gis.content.get(ItemID)
        ydf = gisItem.usage(date_range='1Y', as_df=True)
        if len(ydf.index) > 0:
            ydf = ydf.loc[ydf['Usage']!=0]
            ydf['Year_Views']=ydf['Usage'].sum()
            yViews =(ydf.iloc[0]['Year_Views'])
            lused = ydf['Date'].max()
            ydf = ydf[(ydf['Date'] > lastMonth)]
            if len(ydf.index) > 0:
                ydf['Month_Views']=ydf['Usage'].sum()
                mViews =(ydf.iloc[0]['Month_Views'])
            else:
                mViews = 0
        else:
            yViews = 0
            mViews = 0

        # Add the item record with views to the dataframe
        df = df.append({
        'ItemTitle':str(Item.title.replace(",", "")),
        'ItemID':str(Item.id),
        'ItemType':str(Item.type),
        'Owner':str(Item.owner),
        'Year_Views':yViews,
        'Month_Views':mViews,
        'LastUsed':lused,
        'Last_Searched':tday
        }, ignore_index=True)

        # A little cleanup
        del yViews
        del mViews
        lused = None
        ydf = ydf.iloc[0:0]
    else:
        # print("Item already assessed recently")
        pass

df.drop_duplicates(subset=dataColumns,inplace=True)
df

# Write the Audit Results to CSV

In [ ]:
from tkinter.filedialog import asksaveasfilename
filename = asksaveasfilename(defaultextension='.csv') # show an "Open" dialog box and return the path to the selected file
print(filename)
df.to_csv(filename, index=False)
print("Write data To CSV File Complete!")

# Upload to AGO

In [ ]:
# would be nice to check initially if it exists in AGO and use that to build the df. update when finished

prevCSV = gis.content.search(query="AGOitemUsage", item_type="csv", max_items = 1)
if prevCSV:
    print("Found a previous csv in AGO")
    prevCSV = prevCSV[0]
    display(prevCSV)
    from arcgis.features import FeatureLayerCollection
    testfile_collection = FeatureLayerCollection.fromitem(testfile)
    print("The table already exists")
    
    # left off here- need to create the (hopefully virtual) csv to use to update the service
    testfile_collection.manager.overwrite(csvfile)
    display
else:
    print("We have to upload it")
    csv_item = gis.content.add({}, csvfile)
    display(csv_item)
    csv_lyr = csv_item.publish()
    display(csv_lyr)
print ("The CSV has been published.\nAll done, thanks for playing!")

In [ ]:
df